In [ ]:
# Kütüphaneleri yüklüyoruz.
import cv2
import numpy as np

In [ ]:
# Burada modeli yüklüyoruz. 
from ultralytics import YOLO
# kişinin kask takıp takmadığını bulan model
model= YOLO("helmet.pt") 
# Motor plakasını bulan model
model2= YOLO("moto_plate.pt")

In [ ]:
# plaka okumak için bu modülü kullanacağız
import easyocr
reader = easyocr.Reader(['en']) 


In [6]:


font = cv2.FONT_HERSHEY_SIMPLEX
img=cv2.imread('img4.jpg')
# resmi ilk modele veriyoruz.
results = model(img,verbose=False) 
labels=results[0].names

for i in range(len(results[0].boxes)):
    
    x1,y1,x2,y2=results[0].boxes.xyxy[i]
    score=results[0].boxes.conf[i]
    label=results[0].boxes.cls[i]
    x1,y1,x2,y2,score,label=int(x1),int(y1),int(x2),int(y2),float(score),int(label)
    name=labels[label]
    
    if score<0.7:
        continue
    # eğer kişi kask takıyorsa bu if bloğu çalışır
    # burada kask takan kişi ile takmayan kişi farklı renklerde gösteriliyor
    if name=='HELMET':
        cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),5)
        text=name+' '+str(format(score, '.2f'))
        cv2.putText(img, text,(x1, y1-10), font, 2, (150,0,255), 3)
    # eğer kişi kask takmıyorsa bu if bloğu çalışır
    
    if name=='NOT HELMET':

        cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,0),2)
        text=name+' '+str(format(score, '.2f'))
        cv2.putText(img, text,(x1, y1-10), font, 1.5, (0,0,255), 2)
        # Burada moturun bulunduğu kısmı yeni bir resim olarak tanıladım
        # yeni işlemler burada yapılacak
        img2=img[y1:y2,x1:x2]
        # yeni resmi plaka bulan modele verdik.
        results = model2(img2,verbose=False) 
        labels=results[0].names
        for j in range(len(results[0].boxes)):
            
            x1,y1,x2,y2=results[0].boxes.xyxy[j]
            score=results[0].boxes.conf[j]
            label=results[0].boxes.cls[j]
            x1,y1,x2,y2,score,label=int(x1),int(y1),int(x2),int(y2),float(score),int(label)
            name=labels[label]
            
            if score<0.7:
                continue
            # Burada plakanın olduğu kısmı yeni bir resim olarak tanımladık
            img3=img2[y1:y2,x1:x2]
            # Plaka daha rahat okunsun yeni resmi büyüttük
            img3=cv2.resize(img3,(0,0),fx=3,fy=3)
            # sadece plakanın olduğu kısım olan bu resmi görselden yazı çıkaran bu modele verdik.
            text_result = reader.readtext(img3)
            # burada plakayı ekranda yazdırmak için yeni bir text tanımladık.
            # ikinci kısım plakanın yazı kısmı
            text_plate= 'PLATE: '+text_result[0][-2]
            # Ekrana yazdırırken güzel dursun diye görüntünün sol üst ve sağ üst köşesine arka plan rengi ayarladık.
            img[0:80,0:700]=(102,0,153)
            cv2.putText(img, text_plate,(0, 60), font, 2, (255,255,255), 2)
            
            # Burada plakayı kare içine aldık
            cv2.rectangle(img2,(x1,y1),(x2,y2),(255,0,255),5)
            text=name+' '+str(format(score, '.2f'))
            cv2.putText(img2, text,(x1, y1-10), font, 2, (0,255,255), 3)
  
        
        



        
cv2.imshow("kamera",img)
# Sonuçları kaydetmek için aşağıdaki satırı çalıştırıyoruz.
cv2.imwrite('yeni_image.jpg',img)

cv2.waitKey()
cv2.destroyAllWindows()